<a href="https://colab.research.google.com/github/kilnis/DS-Projects/blob/main/%D0%A2%D0%97_%D0%9A%D1%81%D0%B5%D0%BD%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

# Импорт необходимых библиотек
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, lit

# Инициализация Spark
spark = SparkSession.builder \
    .appName("Example") \
    .getOrCreate()

# Пример создания DataFrame
data = [(1, "Alice"), (2, "Bob"), (3, "Cathy")]
df = spark.createDataFrame(data, ["id", "name"])

# Показать DataFrame
df.show()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=f03774188566b8c7eb7c344e1f9116d26981afc3522dac7cabbbe0c63ce638ea
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark
+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
|  3|Cathy|
+---+-----+



In [3]:
import pyspark

import pyspark.sql.functions as F
from pyspark.sql.functions import col, lit
from pyspark.sql import Window


In [ ]:
import pyspark

from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("test_sql") \
    .getOrCreate()
# spark = init_spark("test_sql", spark_version="2.4.8")
import pyspark.sql.functions as F
from pyspark.sql.functions import col, lit
from pyspark.sql import Window

Обязательные требования к решению:
1) Решение задач необходимо сделать в общем виде, для произвольного набора данных.
2) Решение необходимо выполнить в данном JN с выводом результата на экран.  

## Задача №1

In [14]:
# Набор исходных данных для задачи №1
df = spark.sql("""
select 'Красный' as color union all
select 'Зеленый' as color union all
select 'Синий' as color union all
select 'Желтый' as color union all
select 'Фиолетовый' as color union all
select 'Пурпурный' as color union all
select 'Белый' as color
""")
df.createOrReplaceTempView("table1")
df.show(20,0)

+----------+
|color     |
+----------+
|Красный   |
|Зеленый   |
|Синий     |
|Желтый    |
|Фиолетовый|
|Пурпурный |
|Белый     |
+----------+



In [15]:
comby = spark.sql("""
SELECT a.color AS color_1, b.color AS color_2
FROM table1 a CROSS JOIN table1 b
WHERE a.color < b.color
""")

comby.show(truncate=False)

+---------+----------+
|color_1  |color_2   |
+---------+----------+
|Красный  |Синий     |
|Красный  |Фиолетовый|
|Красный  |Пурпурный |
|Зеленый  |Красный   |
|Зеленый  |Синий     |
|Зеленый  |Фиолетовый|
|Зеленый  |Пурпурный |
|Синий    |Фиолетовый|
|Желтый   |Красный   |
|Желтый   |Зеленый   |
|Желтый   |Синий     |
|Желтый   |Фиолетовый|
|Желтый   |Пурпурный |
|Пурпурный|Синий     |
|Пурпурный|Фиолетовый|
|Белый    |Красный   |
|Белый    |Зеленый   |
|Белый    |Синий     |
|Белый    |Желтый    |
|Белый    |Фиолетовый|
+---------+----------+
only showing top 20 rows



## Задача №2

Есть таблица в которой каждая запись действует определенный период с date_from по date_to.
Необходимо выбрать записи которые попадают в промежуток времени с 2019-03-01 по 2021-03-01
т.е. действовали хотя-бы один день в этом промежутке.
Задачу необходимо решить применяя не более 3 условий фильтрации данных

In [4]:
data = ([("1","2000-01-01","2032-01-02","100"),
         ("2","2010-01-01","2020-01-02","200"),
         ("3","2011-01-01","2020-01-02","300"),
         ("4","2018-01-01","2019-01-02","400"),
         ("5","2019-05-01","2020-05-02","500"),
         ("6","2020-01-01","2030-02-02","600"),
         ("7","2021-02-01","2030-01-02","700")
       ])
columns = ["id","date_from","date_to","atr_a"]
df = spark.createDataFrame(data = data, schema = columns)

df.createOrReplaceTempView("tbl2")
df.show(10,0)

+---+----------+----------+-----+
|id |date_from |date_to   |atr_a|
+---+----------+----------+-----+
|1  |2000-01-01|2032-01-02|100  |
|2  |2010-01-01|2020-01-02|200  |
|3  |2011-01-01|2020-01-02|300  |
|4  |2018-01-01|2019-01-02|400  |
|5  |2019-05-01|2020-05-02|500  |
|6  |2020-01-01|2030-02-02|600  |
|7  |2021-02-01|2030-01-02|700  |
+---+----------+----------+-----+



Фильтрация с помощью SQL:

In [5]:
query = """
SELECT *
FROM tbl2
WHERE date_to >= '2019-03-01' AND date_from <= '2021-03-01'
"""
result_df = spark.sql(query)
result_df.show()

+---+----------+----------+-----+
| id| date_from|   date_to|atr_a|
+---+----------+----------+-----+
|  1|2000-01-01|2032-01-02|  100|
|  2|2010-01-01|2020-01-02|  200|
|  3|2011-01-01|2020-01-02|  300|
|  5|2019-05-01|2020-05-02|  500|
|  6|2020-01-01|2030-02-02|  600|
|  7|2021-02-01|2030-01-02|  700|
+---+----------+----------+-----+



Фильтрация без SQL:

In [13]:
d_start = '2019-03-01'
d_end = '2021-03-01'
res = df.filter((F.col("date_from") <= d_end) & (F.col("date_to") >= d_start))
res.show()

+---+----------+----------+-----+
| id| date_from|   date_to|atr_a|
+---+----------+----------+-----+
|  1|2000-01-01|2032-01-02|  100|
|  2|2010-01-01|2020-01-02|  200|
|  3|2011-01-01|2020-01-02|  300|
|  5|2019-05-01|2020-05-02|  500|
|  6|2020-01-01|2030-02-02|  600|
|  7|2021-02-01|2030-01-02|  700|
+---+----------+----------+-----+



## Задача №3


Есть таблица абонентов с историей изменений. Каждая запись по абоненту действует в пределах периода c date_from по date_to.
Если одна запись закрывается, следующая запись начинается с той же даты.  
По каждому абоненту должна быть только одна действующая запись в интервале [date_from, date_to).
Необходимо написать запрос, который вернет те записи, у которых некорректная дата закрытия.

In [16]:
# Набор исходных данных для задачи №3
data = ([("1","2000-01-01","2000-10-02"),
         ("1","2000-10-02","2020-01-02"),
         ("1","2010-01-01","2019-01-02"),
         ("1","2019-01-02","2023-01-01"),
         ("2","2019-05-01","2020-05-02"),
         ("2","2019-05-01","2020-02-10"),
         ("2","2020-02-10","2020-02-15"),
         ("2","2020-02-16","2030-01-02"),
         ("3","2021-02-01","2022-01-01"),
         ("3","2022-01-01","2023-01-01"),
         ("3","2023-01-01","2030-01-02"),
         ("3","2023-01-01","2024-01-02")
       ])
columns = ["id","date_from","date_to"]
df = spark.createDataFrame(data = data, schema = columns)

df.createOrReplaceTempView("tbl3")
df.show(100,0)

+---+----------+----------+
|id |date_from |date_to   |
+---+----------+----------+
|1  |2000-01-01|2000-10-02|
|1  |2000-10-02|2020-01-02|
|1  |2010-01-01|2019-01-02|
|1  |2019-01-02|2023-01-01|
|2  |2019-05-01|2020-05-02|
|2  |2019-05-01|2020-02-10|
|2  |2020-02-10|2020-02-15|
|2  |2020-02-16|2030-01-02|
|3  |2021-02-01|2022-01-01|
|3  |2022-01-01|2023-01-01|
|3  |2023-01-01|2030-01-02|
|3  |2023-01-01|2024-01-02|
+---+----------+----------+



In [59]:
# tab1 = spark.sql( """
# SELECT id, date_from, date_to,
#     date_to = lag(date_from, -1) over (partition by id ORDER BY id) as correct
# from tbl3
# """)
# tab1.show()

tab2 = spark.sql( """
SELECT id, date_from, date_to
FROM (SELECT id, date_from, date_to,
      date_to = (lag(date_from, -1) over (partition by id ORDER BY id)) as correct
      from tbl3) t
WHERE correct = False
""")

tab2.show()

+---+----------+----------+
| id| date_from|   date_to|
+---+----------+----------+
|  1|2000-10-02|2020-01-02|
|  2|2019-05-01|2020-05-02|
|  2|2020-02-10|2020-02-15|
|  3|2023-01-01|2030-01-02|
+---+----------+----------+



In [49]:
tab1 = spark.sql( """
SELECT id, date_from, date_to,
    LAG(date_to) OVER (PARTITION BY id ORDER BY date_from) AS previous_date_to
FROM tbl3
""")
incorrect_records = tab1.filter(tab1.date_from != tab1.previous_date_to)
.show(100, 0)

+---+----------+----------+----------------+
|id |date_from |date_to   |previous_date_to|
+---+----------+----------+----------------+
|1  |2010-01-01|2019-01-02|2020-01-02      |
|2  |2019-05-01|2020-02-10|2020-05-02      |
|2  |2020-02-16|2030-01-02|2020-02-15      |
|3  |2023-01-01|2024-01-02|2030-01-02      |
+---+----------+----------+----------------+



## Задача №4

Есть две таблицы. В одной хранится история изменений атрибутов абонента. В другой история изменения атрибутов слицевых счета.
Необходимо пересечь данные из этих таблиц таким обазом, чтобы сохранилась история изменения каждого из атрибутов.

In [60]:
data = ([("1","10","2001-01-01","2010-01-01"),
         ("1","20","2010-01-01","2017-01-02"),
         ("1","30","2017-01-01","2020-01-02")
       ])
columns = ["appn","acc","date_from","date_to"]
df_appn = spark.createDataFrame(data = data, schema = columns)
df_appn.createOrReplaceTempView("tbl_appn")

data = ([("10","a","x","2000-01-01","2008-01-01"),
         ("10","b","x","2008-01-01","2009-01-01"),
         ("10","b","y","2009-01-01","2012-01-01"),
         ("20","c","z","2007-01-01","2011-01-02"),
         ("20","c","w","2011-01-02","2020-01-02"),
         ("30","d","w","2015-01-02","2018-01-02"),
         ("30","f","w","2018-01-02","2019-01-02"),
         ("30","f","h","2019-01-02","2021-01-02")
       ])
columns = ["acc","acc_num","tp", "date_from","date_to"]
df_acc = spark.createDataFrame(data = data, schema = columns)
df_acc.createOrReplaceTempView("tbl_acc")

df_appn.show(200,0)
df_acc.show(200,0)

+----+---+----------+----------+
|appn|acc|date_from |date_to   |
+----+---+----------+----------+
|1   |10 |2001-01-01|2010-01-01|
|1   |20 |2010-01-01|2017-01-02|
|1   |30 |2017-01-01|2020-01-02|
+----+---+----------+----------+

+---+-------+---+----------+----------+
|acc|acc_num|tp |date_from |date_to   |
+---+-------+---+----------+----------+
|10 |a      |x  |2000-01-01|2008-01-01|
|10 |b      |x  |2008-01-01|2009-01-01|
|10 |b      |y  |2009-01-01|2012-01-01|
|20 |c      |z  |2007-01-01|2011-01-02|
|20 |c      |w  |2011-01-02|2020-01-02|
|30 |d      |w  |2015-01-02|2018-01-02|
|30 |f      |w  |2018-01-02|2019-01-02|
|30 |f      |h  |2019-01-02|2021-01-02|
+---+-------+---+----------+----------+



In [83]:
history = spark.sql("""
select a.appn, a.acc, b.acc_num, b.tp,
       greatest(a.date_from, b.date_from) as date_from,
       least(a.date_to, b.date_to) as date_to
from tbl_appn a join tbl_acc b on a.acc = b.acc
and greatest(a.date_from, b.date_from) < least(a.date_to, b.date_to)
order by appn, acc, date_from
""")

history.show(truncate=False)

+----+---+-------+---+----------+----------+
|appn|acc|acc_num|tp |date_from |date_to   |
+----+---+-------+---+----------+----------+
|1   |10 |a      |x  |2001-01-01|2008-01-01|
|1   |10 |b      |x  |2008-01-01|2009-01-01|
|1   |10 |b      |y  |2009-01-01|2010-01-01|
|1   |20 |c      |z  |2010-01-01|2011-01-02|
|1   |20 |c      |w  |2011-01-02|2017-01-02|
|1   |30 |d      |w  |2017-01-01|2018-01-02|
|1   |30 |f      |w  |2018-01-02|2019-01-02|
|1   |30 |f      |h  |2019-01-02|2020-01-02|
+----+---+-------+---+----------+----------+

